# Agentic Solution Idea for Business Paint-Point


*What this notebook does*
1. Calls an LLM to pick a **business area** worth exploring.
2. Calls an LLM to surface a sharp **pain‑point** in that area.
3. Calls an LLM to propose an **Agentic AI** solution (roles, tools, loops, risks, quick prototype).
4. Saves a full **JSON trace** of prompts and responses to `trace.json`.

In [ ]:
# Imports & config
import os, json, textwrap
from pathlib import Path
from datetime import datetime
import requests
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
OPENAI_MODEL = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')

# Toggle mock mode automatically if no key is present
DRY_RUN = os.getenv('DRY_RUN', '').lower() in {'1','true','yes'} or not bool(OPENAI_API_KEY)
print('DRY_RUN =', DRY_RUN)

HEADERS = {
    'Authorization': f'Bearer {OPENAI_API_KEY}' if OPENAI_API_KEY else '',
    'Content-Type': 'application/json',
}

DRY_RUN = True


## Prompts
All calls share the same **system prompt**. User prompts are injected with prior outputs.

In [2]:
SYSTEM = {
    'role': 'system',
    'content': (
        'You are a concise, structured strategist. '
        'Always reply as strict JSON only. No prose, no backticks, no explanations. '
        'If lists are needed, use arrays of short strings.'
    ),
}

PROMPTS = {
    'area': (
        "Task: Pick one business area that’s promising for an Agentic AI opportunity.\n"
        "Return JSON with:\n- business_area: string\n- why_promising: string (≤ 2 sentences)\n"
        "Constraints: Keep it under 80 words total."
    ),
    'pain': (
        "Task: In the business area \"{{business_area}}\", identify one high‑value pain‑point.\n"
        "Return JSON with:\n- pain_point: string\n- who_is_affected: string\n- current_workarounds: string\n- why_hard: string\n"
        "Constraints: Keep it under 120 words total."
    ),
    'solution': (
        "Task: Propose an Agentic AI solution for the pain-point \"{{pain_point}}\" in the area \"{{business_area}}\".\n"
        "Return JSON with:\n- solution_name: string\n- agent_roles: string[]\n- data_sources: string[]\n- key_actions: string[]\n- success_metrics: string[]\n- risks: string[]\n- quick_prototype: string[]\n"
        "Constraints: Keep it under 180 words total."
    ),
}

## Minimal OpenAI‑compatible client + helpers
- `chat(messages)` calls the Chat Completions API or returns **mocked** JSON when `DRY_RUN=True`.
- `jparse(s)` safely parses JSON (with a small fallback).

In [3]:
def jparse(s: str):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        s2 = s.strip().strip('`\n ')
        first = s2.find('{'); last = s2.rfind('}')
        if first != -1 and last != -1:
            return json.loads(s2[first:last+1])
        raise

def _mock_response_for(prompt: str) -> str:
    # Heuristic mock based on which prompt it is
    if 'business area' in prompt and 'Return JSON with' in prompt and 'business_area' in prompt:
        return json.dumps({
            'business_area': 'Property Management (SMB)',
            'why_promising': 'Fragmented operations, thin margins, and repetitive workflows enable agents to coordinate vendors and tenants end‑to‑end.'
        })
    if 'identify one high‑value pain‑point' in prompt:
        return json.dumps({
            'pain_point': 'Coordinating maintenance across units: triage, scheduling, and follow‑ups',
            'who_is_affected': 'Property managers and tenants in buildings with 50–500 units',
            'current_workarounds': 'Shared inboxes, spreadsheets, and phone tag with vendors',
            'why_hard': 'Unstructured requests, multi‑party constraints in scheduling, poor status visibility.'
        })
    if 'Propose an Agentic AI solution' in prompt:
        return json.dumps({
            'solution_name': 'FixFlow Agent Mesh',
            'agent_roles': ['Triage Agent','Planner','Vendor Liaison','QA/Evaluator'],
            'data_sources': ['Tenant portal messages','Calendar APIs','Vendor CRM','Photos/Videos'],
            'key_actions': ['Parse issue','Quote ETA','Auto‑schedule vendor','Confirm completion'],
            'success_metrics': ['Time‑to‑schedule','First‑visit fix %','CSAT'],
            'risks': ['Hallucinated scheduling','Vendor no‑shows','Edge cases'],
            'quick_prototype': [
                'Webhook intake from tenant form',
                'LLM triage → category+priority',
                'Heuristic + calendar API scheduling',
                'Email/SMS notifications',
                'Simple dashboard for overrides',
                'Audit log and trace capture'
            ]
        })
    # default fallthrough
    return json.dumps({'note': 'mock'})

def chat(messages):
    if DRY_RUN:
        # Return a plausible mock based on the last user message
        last_user = next((m['content'] for m in reversed(messages) if m['role']=='user'), '')
        return _mock_response_for(last_user)
    url = f"{OPENAI_BASE_URL}/chat/completions"
    resp = requests.post(
        url,
        headers=HEADERS,
        json={
            'model': OPENAI_MODEL,
            'messages': messages,
            'temperature': 0.7,
        },
        timeout=60,
    )
    resp.raise_for_status()
    data = resp.json()
    return data['choices'][0]['message']['content'].strip()

## Run the three‑call flow
This cell performs the full chain and writes `trace.json`.

In [4]:
def run_chain():
    trace = {'calls': [], 'created_at': datetime.utcnow().isoformat() + 'Z'}
    messages = [SYSTEM]

    # 1) Business area
    messages.append({'role': 'user', 'content': PROMPTS['area']})
    raw1 = chat(messages)
    out1 = jparse(raw1)
    trace['calls'].append({'step': 1, 'prompt': PROMPTS['area'], 'response': out1})

    # 2) Pain‑point
    p2 = PROMPTS['pain'].replace('{{business_area}}', out1['business_area'])
    messages.append({'role': 'user', 'content': p2})
    raw2 = chat(messages)
    out2 = jparse(raw2)
    trace['calls'].append({'step': 2, 'prompt': p2, 'response': out2})

    # 3) Agentic solution
    p3 = PROMPTS['solution'] \
        .replace('{{business_area}}', out1['business_area']) \
        .replace('{{pain_point}}', out2['pain_point'])
    messages.append({'role': 'user', 'content': p3})
    raw3 = chat(messages)
    out3 = jparse(raw3)
    trace['calls'].append({'step': 3, 'prompt': p3, 'response': out3})

    # Save trace
    with open('trace.json', 'w') as f:
        json.dump(trace, f, indent=2)

    return out1, out2, out3

area, pain, solution = run_chain()
print('=== Result ===')
print('Business Area:', area.get('business_area'))
print('Pain‑Point   :', pain.get('pain_point'))
print('Solution     :', solution.get('solution_name'))

=== Result ===
Business Area: Property Management (SMB)
Pain‑Point   : Coordinating maintenance across units: triage, scheduling, and follow‑ups
Solution     : FixFlow Agent Mesh


## Inspect the trace.json

In [5]:
with open('trace.json') as f:
    trace = json.load(f)
print(json.dumps(trace, indent=2))

{
  "calls": [
    {
      "step": 1,
      "prompt": "Task: Pick one business area that\u2019s promising for an Agentic AI opportunity.\nReturn JSON with:\n- business_area: string\n- why_promising: string (\u2264 2 sentences)\nConstraints: Keep it under 80 words total.",
      "response": {
        "business_area": "Property Management (SMB)",
        "why_promising": "Fragmented operations, thin margins, and repetitive workflows enable agents to coordinate vendors and tenants end\u2011to\u2011end."
      }
    },
    {
      "step": 2,
      "prompt": "Task: In the business area \"Property Management (SMB)\", identify one high\u2011value pain\u2011point.\nReturn JSON with:\n- pain_point: string\n- who_is_affected: string\n- current_workarounds: string\n- why_hard: string\nConstraints: Keep it under 120 words total.",
      "response": {
        "pain_point": "Coordinating maintenance across units: triage, scheduling, and follow\u2011ups",
        "who_is_affected": "Property managers